# The Genesis process

Creating a new game requires the creation of a system and planets, factions and people.

This notebook tests that process as well as documents it. 

In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

%load_ext lab_black

In [2]:
from app import creators
from app import objects

In [3]:
from app.creators import homeworld
from app.creators import universe

An example `data` set for testing. This is the user form that the user submits when creating a new game. 

You can get this from the template at `app\templates\app\creation\genesis.js`


In [4]:
data = {
    "label": "form",
    "name": "worldgenform",
    "objid": "0000000000001",
    "owner": "user.username",
    "username": "user.username",
    "accountid": "0000000000001",
    "conformity": 0.5,
    "constitution": 0.5,
    "literacy": 0.5,
    "aggression": 0.5,
    "num_planets": 4,
    "num_moons": 10,
    "starting_pop": 7,
    "organics": 0.5,
    "minerals": 0.5,
}

You'll also need to configuration files.

In [5]:
conf = creators.universe.configurations.get_configurations()

Each object as an `__init__` function that creates it, however not all data is spawed on init. 

## The Solar System and Planets

In [6]:
system = universe.celestials.System(data)
system

<sytem: ordered; 5982233900478; Troeldanmai>

In [7]:
system.get_data()

{'objid': '5982233900478',
 'name': 'Troeldanmai',
 'label': 'sytem',
 'class': 'ordered',
 'isHomeSystem': True,
 'glat': -5.286,
 'glon': 33.355,
 'gelat': 0.893}

Each object as an `__init__` function that creates it, and populates it with data available at that time. Celestial objects have dependancies on other objects. 

In [8]:
star = objects.celestials.Star(conf["star_config"], system)
star.get_data()

{'name': 'Laskou',
 'class': 'G',
 'objid': '1793066210450',
 'label': 'star',
 'radius': 106}

Each object also inherits a `get_fundamentals()` method that ensures that needed default values are present. `get_data()` extends that functionality. This ensures that objects can alwasy interact with the graph. 

In [9]:
star.get_fundimentals()

{'name': 'Laskou', 'class': 'G', 'objid': '1793066210450', 'label': 'star'}

Procedurally generated planets chose from a list of potential types in the `conf`. To force a particular kind of outcome, reduce the options in the configuration. 

In [10]:
terrestrial_config = {"terrestrial": conf["planet_config"]["terrestrial"]}
home_planet = objects.celestials.Planet(conf=terrestrial_config, orbiting=star)
home_planet.get_data()

{'name': 'Hefer',
 'class': 'terrestrial',
 'objid': '5437091911939',
 'label': 'planet',
 'radius': 0.742,
 'mass': 0,
 'orbitsDistance': 0.64,
 'orbitsId': '1793066210450',
 'orbitsName': 'Laskou',
 'isSupportsLife': False,
 'isPopulated': False}

Creating a group of planets using list comprehension. Note that celestial objects have a custom `__repr__` function that makes them easy to manage.

In [11]:
planets = [
    objects.celestials.Planet(conf=conf["planet_config"], orbiting=star)
    for p in range(int(data["num_planets"]) - 1)
]
planets

[<planet: ice; 5914020021432; Ferhinbeachcon>,
 <planet: gas; 2773891649589; Dongdespemel>,
 <planet: gas; 8614847297599; Gnaper>]

In [12]:
moons = [
    objects.celestials.Moon(conf["moon_config"], planets)
    for p in range(int(data["num_moons"]))
]
moons

[<moon: terrestrial; 3353062424749; Dahepest>,
 <moon: rocky; 1890146811341; Turstahas>,
 <moon: ice; 7595218875152; Thasiagon>,
 <moon: rocky; 5514871583005; Tur>,
 <moon: ice; 3122366938012; Bileshan>,
 <moon: rocky; 0403695317295; Anbai>,
 <moon: ice; 8950016618674; Reigoulas>,
 <moon: rocky; 1135803498045; Grankra>,
 <moon: rocky; 6660072627777; Zi>,
 <moon: ice; 0281489707607; Snores>]

Getting the nodes and edges of each by calling the `self.orbiting` propperty.

In [13]:
moons[0].orbiting

<planet: gas; 2773891649589; Dongdespemel>

In [14]:
home_planet.orbiting

<star: G; 1793066210450; Laskou>

Additionally, you can quickly navigate the system by referencing other objects. 

In [15]:
home_planet.orbiting.system

<sytem: ordered; 5982233900478; Troeldanmai>

Getting the nodes for the graph. Sandwich all of the items together and get the data using the same generic function.

In [16]:
all_entities = [system] + [star] + moons + planets + [home_planet]
all_nodes = [b.get_data() for b in all_entities]
pd.DataFrame(all_nodes)

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,orbitsName,orbitsDistance,mass,isSupportsLife,isPopulated
0,5982233900478,Troeldanmai,sytem,ordered,True,-5.286,33.355,0.893,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1793066210450,Laskou,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,3353062424749,Dahepest,moon,terrestrial,NaN,NaN,NaN,NaN,0.703429,2773891649589,Dongdespemel,4.0001,0.009625,False,False
3,1890146811341,Turstahas,moon,rocky,NaN,NaN,NaN,NaN,0.095947,8614847297599,Gnaper,4.0660,0.000228,False,False
4,7595218875152,Thasiagon,moon,ice,NaN,NaN,NaN,NaN,0.162808,5914020021432,Ferhinbeachcon,3.9710,0.010918,False,False
5,5514871583005,Tur,moon,rocky,NaN,NaN,NaN,NaN,0.032672,5914020021432,Ferhinbeachcon,3.9371,0.000343,False,False
6,3122366938012,Bileshan,moon,ice,NaN,NaN,NaN,NaN,0.110778,8614847297599,Gnaper,4.0001,0.005138,False,False
7,0403695317295,Anbai,moon,rocky,NaN,NaN,NaN,NaN,0.667011,8614847297599,Gnaper,4.0001,0.000481,False,False
8,8950016618674,Reigoulas,moon,ice,NaN,NaN,NaN,NaN,0.195556,5914020021432,Ferhinbeachcon,4.1110,0.019650,False,False
9,1135803498045,Grankra,moon,rocky,NaN,NaN,NaN,NaN,0.408325,5914020021432,Ferhinbeachcon,3.9520,0.000485,False,False


Getting the edge values to update the graph: 

In [17]:
orbiting_bodies = [home_planet] + planets + moons

orbiting_edges = [i.get_orbits_edge() for i in orbiting_bodies]

pd.DataFrame(orbiting_edges)

,node1,node2,label,orbit_distance
0,5437091911939,1793066210450,orbits,0.6400
1,5914020021432,1793066210450,orbits,19.1410
2,2773891649589,1793066210450,orbits,8.4490
3,8614847297599,1793066210450,orbits,6.9280
4,3353062424749,2773891649589,orbits,0.0001
5,1890146811341,8614847297599,orbits,0.0660
6,7595218875152,5914020021432,orbits,0.0340
7,5514871583005,5914020021432,orbits,0.0001
8,3122366938012,8614847297599,orbits,0.0001
9,0403695317295,8614847297599,orbits,0.0001


Check the orbiting logic

In [18]:
[
    f"{i.type} {i.label} orbits a {i.orbiting.type} {i.orbiting.label}"
    for i in orbiting_bodies
]

['terrestrial planet orbits a G star',
 'ice planet orbits a G star',
 'gas planet orbits a G star',
 'gas planet orbits a G star',
 'terrestrial moon orbits a gas planet',
 'rocky moon orbits a gas planet',
 'ice moon orbits a ice planet',
 'rocky moon orbits a ice planet',
 'ice moon orbits a gas planet',
 'rocky moon orbits a gas planet',
 'ice moon orbits a ice planet',
 'rocky moon orbits a ice planet',
 'rocky moon orbits a gas planet',
 'ice moon orbits a gas planet']

In [19]:
system_bodies = orbiting_bodies + [star]

system_edges = [i.get_in_system_edge() for i in system_bodies]

pd.DataFrame(system_edges)

,node1,node2,label
0,5437091911939,5982233900478,isIn
1,5914020021432,5982233900478,isIn
2,2773891649589,5982233900478,isIn
3,8614847297599,5982233900478,isIn
4,3353062424749,5982233900478,isIn
5,1890146811341,5982233900478,isIn
6,7595218875152,5982233900478,isIn
7,5514871583005,5982233900478,isIn
8,3122366938012,5982233900478,isIn
9,0403695317295,5982233900478,isIn


### Scanning the homeworld
the homeworld already has some resources known. `scan_body()` is inherited by the base object. So everyone should be able to do it. 

In [20]:
home_planet.scan_body()
home_planet.resources

[<resource: 7027075589522; Organic>,
 <resource: 5013945634713; Common Minerals>,
 <resource: 9561165380113; Rare Minerals>,
 <resource: 6331644192322; Water>]

In [21]:
pd.DataFrame([i.get_data() for i in home_planet.resources])

,name,objid,label,volume,max_volume,description,replenish_rate
0,Organic,7027075589522,resource,931,931,bilogical material that can be consumed by pops,10.0
1,Common Minerals,5013945634713,resource,93,93,Iron and other common material used in constru...,NaN
2,Rare Minerals,9561165380113,resource,47,47,"lithium, silver and other rare minerals used i...",NaN
3,Water,6331644192322,resource,10842,10842,"H2O ready to be consumed, either frozen or in ...",NaN


In [22]:
pd.DataFrame([i.get_location_edge() for i in home_planet.resources])

,node1,node2,label
0,5437091911939,7027075589522,has
1,5437091911939,5013945634713,has
2,5437091911939,9561165380113,has
3,5437091911939,6331644192322,has


In [23]:
[
    f"{i.location.type} {i.location.label} has {i.volume} {i.name}"
    for i in home_planet.resources
]

['terrestrial planet has 931 Organic',
 'terrestrial planet has 93 Common Minerals',
 'terrestrial planet has 47 Rare Minerals',
 'terrestrial planet has 10842 Water']

# The Full Automated Process

Usefull as QA, to ensure that the process will run end-to-end

In [24]:
homesystem_data = universe.build_homeSystem(data, username="notebook")
pd.DataFrame(homesystem_data["nodes"])

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,...,accountid,conformity,constitution,literacy,aggression,num_planets,num_moons,starting_pop,organics,minerals
0,9906858073848,Hohesrou,sytem,ordered,True,12.351,19.232,-2.46,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6348716548791,Liang,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7042221163618,Koy,moon,terrestrial,NaN,NaN,NaN,NaN,0.028690,0844482884443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3038898074903,Ciubersu,moon,rocky,NaN,NaN,NaN,NaN,0.096343,9427771377020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0439974430248,Marnor,moon,terrestrial,NaN,NaN,NaN,NaN,0.129710,9427771377020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6652065045638,Nalkreispon,moon,rocky,NaN,NaN,NaN,NaN,0.023838,9427771377020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1919779447481,Jiang,moon,terrestrial,NaN,NaN,NaN,NaN,0.110619,9427771377020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7064298700279,Roybo,moon,ice,NaN,NaN,NaN,NaN,0.004913,0844482884443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2006832722085,Kin,moon,rocky,NaN,NaN,NaN,NaN,0.005822,0844482884443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6588739051411,Newgro,moon,rocky,NaN,NaN,NaN,NaN,0.102025,2743619409348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
print(
    f"{len(homesystem_data['edges'])}  total edges for {len(homesystem_data['nodes'])} objects"
)
pd.DataFrame(homesystem_data["edges"]).groupby("label").count()

37  total edges for 22 objects


,node1,node2,orbit_distance
label,,,
createdFrom,1,1,0
has,4,4,0
isIn,16,16,0
orbits,15,15,15
submitted,1,1,0


### Rulse For Ontology:
here is a point to check that the ontology matches this format:
* everything camelCase (starting on lower)
* Don't reference the subject in the edge label (has not hasResource)
* Every node is a Class that can be found in `objects`
* Form and Account are the exception

# The planet Surface

In [ ]:
homeworld_data = homeworld.build_people(data)
homeworld_data

([{'name': 'Badla',
   'objid': '2152333961637',
   'label': 'species',
   'consumes': ['Organic'],
   'effuses': ['Organic waste', 'Plastics'],
   'viral_resilience': 0.7,
   'habitat_resilience': 0.2},
  {'name': 'Fucas Hyjiangfuca',
   'objid': '6714538532858',
   'label': 'pop',
   'conformity': 0.574,
   'literacy': 0.277,
   'aggression': 0.487,
   'constitution': 0.577,
   'health': 0.5,
   'isInFaction': '8424616458051',
   'industry': 0.532,
   'wealth': 0.4045,
   'factionLoyalty': 0.655,
   'isIdle': 'true'},
  {'name': 'Rosar Orbarnu',
   'objid': '8368216969422',
   'label': 'pop',
   'conformity': 0.606,
   'literacy': 0.605,
   'aggression': 0.553,
   'constitution': 0.547,
   'health': 0.5,
   'isInFaction': '8422192744305',
   'industry': 0.55,
   'wealth': 0.5775,
   'factionLoyalty': 0.778,
   'isIdle': 'true'},
  {'name': 'Rosar Cor',
   'objid': '2956170443012',
   'label': 'pop',
   'conformity': 0.604,
   'literacy': 0.526,
   'aggression': 0.597,
   'constitutio